<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re

# Create Player dataset

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

# Create custom projections
Maybe delete later when real projections available

In [3]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(25)

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,Christian McCaffrey,9,RB,1.0,21.234806,20.328316,17.543452,22.662834,25.959152,22.179362,...,21.965841,0.000000,20.444853,22.284246,23.344235,21.056406,22.192989,22.306585,24.911723,20.940504
1,CeeDee Lamb,7,WR,2.6,19.604216,21.406900,17.749509,25.311848,20.235568,24.368999,...,20.784332,23.802523,22.225155,23.187785,21.681881,23.275003,23.008944,22.996130,24.526137,24.739593
2,Tyreek Hill,6,WR,3.2,22.710703,21.861307,19.172565,22.205220,21.573223,0.000000,...,24.176103,21.627182,26.680898,28.643921,20.999092,22.563823,19.385338,21.353073,19.521063,21.106581
3,Bijan Robinson,12,RB,5.0,18.280054,21.157249,22.675798,19.030311,23.311529,18.694860,...,23.699484,19.918981,21.106497,23.597839,0.000000,22.220715,20.857229,24.483343,26.524289,19.094594
4,Breece Hall,12,RB,5.4,24.016473,24.329575,24.468796,21.094430,20.868339,25.104266,...,19.205546,23.921637,21.408559,21.691388,0.000000,21.403170,18.500827,21.047284,24.573071,18.240446
5,Amon-Ra St. Brown,5,WR,6.2,22.095168,22.347731,21.547792,20.314170,0.000000,23.648298,...,20.231943,22.701058,19.760384,22.949006,22.180090,22.889823,20.972332,21.553522,24.146418,21.546979
6,Ja'Marr Chase,12,WR,6.6,21.120805,22.163105,24.682779,20.308055,23.291196,19.513181,...,20.224000,20.671229,23.089219,18.343613,0.000000,20.452984,19.136878,23.227981,21.511224,22.984648
7,Justin Jefferson,6,WR,7.0,19.983632,21.016821,20.074235,22.836640,19.847931,0.000000,...,19.307650,23.910831,18.470803,18.648840,21.967521,23.090831,18.079711,19.690238,21.129639,23.515908
8,Saquon Barkley,5,RB,9.2,21.179989,19.645498,21.294555,13.858546,0.000000,22.052066,...,21.157099,22.327497,18.380606,22.814193,22.940100,25.420679,23.835737,24.422317,24.566482,22.311903
9,A.J. Brown,5,WR,10.2,19.657829,21.109776,19.807157,20.559331,0.000000,21.894687,...,18.980740,19.054354,19.502581,22.028694,23.730866,22.041204,22.378043,21.575997,20.183041,20.425741


In [4]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [7]:
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

# --- 1. Parameter (aus deinem paste-2.txt) ---
players        = df_small['Player'].tolist()
positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks          = list(range(1,18))
pos            = dict(zip(df_small['Player'], df_small['POS']))
pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
beta           = {t:100.0 for t in weeks}
gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
alpha, lambda_0, lambda_1, lambda_2 = 1.0, 1, 100, 150
df_sorted = df_small.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1

# --- 2. Teams, DM-Team und Snake-Draft ---
teams          = [f"Team {i+1}" for i in range(6)]
DM_team        = "Team 4"
num_rounds     = 15
draft_order    = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

# --- 3. Initialisierung ---
rosters        = {tm: [] for tm in teams}
available      = set(players)
draft_log      = []

# --- 4. Hauptschleife über alle Picks ---
for pick_idx, team in enumerate(draft_order, start=1):
    # 4.1 Ranking der verbleibenden Spieler aktualisieren
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
    Rk  = {p: i+1 for i,p in enumerate(rem)}

    # 4.2 DM-Pick via MIP
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # Variablen nur für verbleibende Spieler
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}") for i in rem}
        x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
             for i in rem for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # Fixiere vergangene Picks
        for p in rosters[team]:
            m += y[p] == 1

        # Objective
        m.objective = (
            lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
          + lambda_1 * xsum(z[t] for t in weeks[:15])
          + lambda_2 * xsum(z[t] for t in weeks[15:])
        )

        # Position‐ und Roster‐Constraints
        for j in positions:
            # Müssen mindestens gamma[j] ziehen
            m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
            for t in weeks:
                m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]
        # maximale Picks pro Team
        m += xsum(y[i] for i in rem) <= num_rounds
        # nur gedraftete Spieler in der Week‐Lineup
        for i in rem:
            for t in weeks:
                m += x[i,t] <= y[i]
        # Win‐Indicator
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

        # Robuste Draft‐Constraint (1b) – grob implementiert
        n_k = pick_idx
        for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
            top_cut = int(alpha*(future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i,r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

        m.optimize()
        # gewählten Spieler extrahieren
        chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
    else:
        topk = max(1, len(rem)//20)  # 5% als Beispiel
        pick = random.choice(rem[:topk])

    # 4.4 Update
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx, "Team": team, "Player": pick,
        "Round": (pick_idx-1)//len(teams)+1
    })

# --- 5. Ergebnis als DataFrame ---
df_draft = pd.DataFrame(draft_log)
print(df_draft)


KeyError: 'James Cook'